Importing Python libraries

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

## Action Plan 

Here we are joining two different data. 
The data are as follows 

1. strike_data_clean1.csv
    
2. VCW_weather_data.csv


Reading this data

In [2]:
df_strike = pd.read_csv("./data/strike_data_clean_1.csv")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (10,11,33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_weather = pd.read_excel("./data/weather_data.xlsx" , sheet_name = 'IAD')

Performing Left Join of Wildlife strike data on Weather data 

In [4]:
def prepare_weather_data(df):
    
    ### Clean Datetime Column
    df['datetime'] = pd.to_datetime(df['datetime']).dt.strftime('%Y-%m-%d')
    
    ## Taking mean at daily level
    df_daily = df.groupby("datetime").mean().reset_index()
    
    return df_daily

In [5]:
def prepare_strike_data(df , airport_id = 'KIAD'):
    df = df[df['AIRPORT_ID'] == airport_id]
    df['INCIDENT_DATE'] = pd.to_datetime(df['INCIDENT_DATE']).dt.strftime('%Y-%m-%d')
    df = df.reset_index(drop =True)
    
    return df

In [6]:
df_weather = prepare_weather_data(df_weather)
df_strike_IAD = prepare_strike_data(df_strike , airport_id = 'KIAD')

<ipython-input-5-3ba02197ed27>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INCIDENT_DATE'] = pd.to_datetime(df['INCIDENT_DATE']).dt.strftime('%Y-%m-%d')


In [7]:
df_strike_IAD.shape

(2426, 34)

In [8]:
df_weather.shape

(1384, 21)

In [10]:
# df_strike_IAD

In [11]:
df_merge_ws = df_weather.merge( right = df_strike_IAD 
                                    , how = "left"
                                    , left_on = ['datetime']
                                    , right_on = ['INCIDENT_DATE'])

In [28]:
df_merge_ws['STRIKE_COUNT'] = df_merge_ws.INDEX_NR.apply(lambda x: 0 if pd.isnull(x) else 1)

In [45]:
df_merge_ws.to_csv('df_merge_ws_2.csv', index=False)

In [32]:
# df_merge_ws.to_csv('df_merge_2.csv', index=False)

df_merge_ws[['temp', 'feelslike', 'humidity', 'dew', 'precip', 'precipprob', 'snow', 'snowdepth', 'windgust', 'windspeed',
       'winddir', 'pressure', 'visibility', 'cloudcover', 'solarradiation',
       'solarenergy', 'uvindex', 'tzoffset', 'severerisk' , 'STRIKE_COUNT']].corr()[['STRIKE_COUNT']].to_csv("correlation.csv" )

In [36]:
numeric_columns = df_merge_ws[['temp', 'feelslike', 'humidity', 'dew', 'precip', 'precipprob', 'snow', 'snowdepth', 'windgust', 'windspeed',
       'winddir', 'pressure', 'visibility', 'cloudcover', 'solarradiation',
       'solarenergy', 'uvindex', 'tzoffset', 'severerisk' , 'STRIKE_COUNT']].corr()[['STRIKE_COUNT']]

In [43]:
correlation_matrix = numeric_columns.unstack().sort_values(ascending=False, key= abs)


In [46]:
df_weather.datetime.min() , df_weather.datetime.max()

('2020-01-01', '2023-10-15')

In [2]:
#df_merge.to_csv('df_merge_ws.csv', index=False)